# Course project Recommendation Systems


## **Основное**
- Дедлайн - 4 апреля 23:59
- Целевая метрика precision@5
- Бейзлайн решения - [MainRecommender](https://github.com/geangohn/recsys-tutorial/blob/master/src/recommenders.py)
- Сдаем ссылку на github с решением. В решении должны быть отчетливо видна метрика на новом тестовом сете из файла retail_test1.csv, то есть вам нужно для всех юзеров из этого файла выдать выши рекомендации, и посчитать на actual покупках precision@5. 

**Hint:** Мы не рассматриваем холодный старт для пользователя, все наши пользователя одинаковы во всех сетах, поэтому нужно позаботиться об их исключении из теста.

---

# Загрузка библиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items, popularity_recommendation, postfilter_items
from recommenders import MainRecommender

pd.pandas.set_option('display.max_columns', None)
import warnings
warnings.simplefilter('ignore')

import statistics

## Загрузка данных

In [2]:
# Загрузка данных

PATH_DATA = "data"
data = pd.read_csv(os.path.join(PATH_DATA,'retail_train.csv'))
item_features = pd.read_csv(os.path.join(PATH_DATA,'product.csv'))
user_features = pd.read_csv(os.path.join(PATH_DATA,'hh_demographic.csv'))

# Преобразование столбцов
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns = {'product_id': 'item_id'}, inplace = True)
user_features.rename(columns = {'household_key': 'user_id'}, inplace = True)

# Проверка item_features

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


---

## Разбиение на train, valid, test

In [3]:
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

#данные для обучения первой модели
data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]

#данные для валидации первой модели
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

#данные для обучения второй модели
data_train_lvl_2 = data_val_lvl_1.copy() 

#данные для теста второй модели
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

In [4]:
# Предобработка данных

n_items_before = data_train_lvl_1['item_id'].nunique()
data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features = item_features, take_n_popular = 5000)
n_items_after = data_train_lvl_1['item_id'].nunique()

print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [5]:
# Cold to warm start

def print_stats_data(df_data, name_df):
    
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data['user_id'].nunique()} Items: {df_data['item_id'].nunique()}")


In [6]:
# Общие пользователи

common_users = list(set(data_train_lvl_1.user_id.values)&(set(data_val_lvl_1.user_id.values))&set(data_val_lvl_2.user_id.values))

data_train_lvl_1 = data_train_lvl_1[data_train_lvl_1.user_id.isin(common_users)]
data_val_lvl_1 = data_val_lvl_1[data_val_lvl_1.user_id.isin(common_users)]
data_train_lvl_2 = data_train_lvl_2[data_train_lvl_2.user_id.isin(common_users)]
data_val_lvl_2 = data_val_lvl_2[data_val_lvl_2.user_id.isin(common_users)]

print_stats_data(data_train_lvl_1,'train_lvl_1')
print_stats_data(data_val_lvl_1,'val_lvl_1')
print_stats_data(data_train_lvl_2,'train_lvl_2')
print_stats_data(data_val_lvl_2,'val_lvl_2')

train_lvl_1
Shape: (998901, 13) Users: 1915 Items: 5001
val_lvl_1
Shape: (163261, 12) Users: 1915 Items: 27118
train_lvl_2
Shape: (163261, 12) Users: 1915 Items: 27118
val_lvl_2
Shape: (115989, 12) Users: 1915 Items: 24042


---

## Обучение первого уровня

In [7]:
# Создание модели

recommender = MainRecommender(data_train_lvl_1)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [8]:
# Кандидаты для второго уровня

result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns = ['user_id', 'actual']

result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,6,"[1024306, 1102949, 6548453, 835394, 940804, 96..."


In [9]:
result_lvl_1['als_recommendations'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=50))
result_lvl_1['own_recommendations'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
result_lvl_1['similar_items_recommendation'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(x, N=50))
result_lvl_1['similar_users_recommendation'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_users_recommendation(x, N=50))
result_lvl_1['popular_recommendation'] = result_lvl_1['user_id'].apply(lambda x: popularity_recommendation(data_train_lvl_1, n=5))

result_lvl_1.head(2)

,user_id,actual,als_recommendations,own_recommendations,similar_items_recommendation,similar_users_recommendation,popular_recommendation
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[1082212, 9296957, 6534030, 975633, 1104349, 9...","[856942, 9297615, 5577022, 1074612, 9655212, 1...","[824758, 1022053, 1007512, 990656, 5577022, 94...","[854133, 869868, 6534201, 9677466, 953539, 879...","[999999, 1070702, 933835, 1065538, 863447]"
1,6,"[1024306, 1102949, 6548453, 835394, 940804, 96...","[1042616, 1037337, 1084036, 928146, 871611, 94...","[13003092, 995598, 5570685, 6979393, 1084036, ...","[948650, 895166, 1040346, 10455579, 8203720, 8...","[1056212, 970160, 897611, 839346, 905436, 9701...","[999999, 1070702, 933835, 1065538, 863447]"


In [10]:
def calc_recall(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: recall_at_k(row[col_name], row['actual'], k = top_k), axis = 1).mean()
        
def calc_precision(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: precision_at_k(row[col_name], row['actual'], k = top_k), axis = 1).mean()

In [11]:
# Recall@50 для полученных рекомендаций

sorted(calc_recall(result_lvl_1, 50), key = lambda x: x[1],reverse = True)

[('own_recommendations', 0.063584522635807),
 ('als_recommendations', 0.03830033117572439),
 ('similar_items_recommendation', 0.022113220990733862),
 ('similar_users_recommendation', 0.00714078083021261),
 ('popular_recommendation', 0.001902219208505204)]

In [12]:
# Pricision@5 для полученных рекомендаций

sorted(calc_precision(result_lvl_1, 5), key = lambda x: x[1],reverse = True)

[('own_recommendations', 0.21305483028720626),
 ('als_recommendations', 0.09107049608355092),
 ('similar_items_recommendation', 0.048772845953002616),
 ('popular_recommendation', 0.028302872062663187),
 ('similar_users_recommendation', 0.012950391644908618)]

---

## Обучение второго уровня

In [13]:
# Берем пользователей из data_train_lvl_2 для ранжирования

users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

# Подбираем для каждого пользователя кандидатов в рекомендации

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N = 50))

In [14]:
# Каждый кандидат должен быть один в строке

s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis = 1).stack().reset_index(level = 1, drop = True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis = 1).join(s)

users_lvl_2.head(4)

,user_id,item_id
0,2070,834103
0,2070,1119399
0,2070,944588
0,2070,1107824


In [15]:
# Проверка warm start

users_lvl_2['user_id'].nunique()

1915

In [16]:
# Датасет для ранжирования (для обучения модели второго уровня) на каждого юзера по 50 кандидатов

targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()

targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on = ['user_id', 'item_id'], how = 'left')

targets_lvl_2['target'].fillna(0, inplace = True)

targets_lvl_2.head(4)

,user_id,item_id,target
0,2070,834103,1.0
1,2070,834103,1.0
2,2070,834103,1.0
3,2070,834103,1.0


In [17]:
# Подготовка описательных фичей (фичи для user_id)

def new_user_features(data, user_features, item_features):
    
    new_user_features = user_features.merge(data, on = 'user_id', how = 'left')
    new_item_features = data.merge(item_features, on = 'item_id', how = 'left')
    
    # Средний чек
    average_check = new_user_features.groupby(['user_id'])['sales_value'].mean().reset_index()
    average_check.rename(columns = {'sales_value': 'average_sales_value'}, inplace = True)
    user_features = user_features.merge(average_check)

     
    # Количество уникальных категорий покупателя
    num_unique_department = new_item_features.groupby(['user_id'])['department'].nunique().reset_index()
    num_unique_department.rename(columns = {'department': 'num_unique_department'}, inplace = True)
    user_features = user_features.merge(num_unique_department, on = 'user_id', how = 'left')
    
   
    # Количество покупок в каждой категории
    num_sales_in_category = new_item_features[['user_id', 'quantity', 'department']].groupby(['user_id', 'department']).sum().reset_index()
    num_sales_in_category.rename(columns = {'quantity': 'num_quantity'}, inplace = True) 
    num_sales_in_category.drop(labels = [0], axis = 0, inplace = True)

    users_num_cat_sales_dict = {}
    for user_id in num_sales_in_category['user_id'].unique():
        users_num_cat_sales_dict[user_id] = dict(list(zip(num_sales_in_category[num_sales_in_category['user_id'] == user_id]['department'].values, \
                                                      num_sales_in_category[num_sales_in_category['user_id'] == user_id]['num_quantity'].values)))

    for cat in item_features['department'].unique():
        new_item_features[f'num_sales_in_{cat}'] = 0

    for user_id in new_item_features['user_id'].unique():
        for cat in users_num_cat_sales_dict[user_id].keys():
            new_item_features.loc[(new_item_features['user_id'] == user_id) & (new_item_features['department'] == cat), f'num_sales_in_{cat}']=\
            users_num_cat_sales_dict[user_id][cat]

    feat_to_merge = ['user_id'] + new_item_features.columns.tolist()[18:]
    user_num_sales_in_cat = new_item_features[feat_to_merge]
    user_num_sales_in_cat = user_num_sales_in_cat.groupby('user_id').max().reset_index()
    user_features = user_features.merge(user_num_sales_in_cat, on = 'user_id', how = 'left')
    
    user_features = user_features.drop('num_sales_in_ ', axis = 1)
    
    # Среднее время покупки пользователем
    new_user_features['hour'] = new_user_features['trans_time'] // 100
    hour = new_user_features.groupby('user_id')['hour'].median().reset_index()
    hour.columns = ['user_id', 'median_sales_hour_for_user']
    user_features = user_features.merge(hour, on = 'user_id', how = 'left')
    
    # Популярный день недели покупки юзера
    new_user_features['weekday'] = new_user_features['day'] % 7
    week_day = new_user_features.groupby('user_id')['weekday'].apply(statistics.mode).reset_index()
    week_day.columns = ['user_id', 'mode_sales_day_for_user']
    user_features = user_features.merge(week_day, on = 'user_id', how = 'left')
    
    # Cредний чек корзины покупателя
    average_check_bascket = new_user_features.groupby(['user_id', 'basket_id'])['sales_value'].sum().reset_index()
    average_check_bascket = average_check_bascket.groupby('user_id')['sales_value'].mean().reset_index()
    average_check_bascket.columns = ['user_id', 'mean_basket_check']
    user_features = user_features.merge(average_check_bascket, on = 'user_id', how = 'left')
    
    # Кол-во уникальных товаров, купленных клиентом
    item_unique = new_user_features.groupby(['user_id'])['item_id'].nunique().reset_index()
    item_unique.columns = ['user_id', 'n_items']
    user_features = user_features.merge(item_unique, on = 'user_id', how = 'left')
    
    user_features = user_features.replace(np.nan, 0)
    
    return user_features

In [18]:
%%time

user_features_new = new_user_features(data_train_lvl_2, user_features, item_features)
user_features_new.head(3)

CPU times: user 1min 40s, sys: 797 ms, total: 1min 41s
Wall time: 1min 42s


,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id,average_sales_value,num_unique_department,num_sales_in_GROCERY,num_sales_in_MISC. TRANS.,num_sales_in_PASTRY,num_sales_in_DRUG GM,num_sales_in_MEAT-PCKGD,num_sales_in_SEAFOOD-PCKGD,num_sales_in_PRODUCE,num_sales_in_NUTRITION,num_sales_in_DELI,num_sales_in_COSMETICS,num_sales_in_MEAT,num_sales_in_FLORAL,num_sales_in_TRAVEL & LEISUR,num_sales_in_SEAFOOD,num_sales_in_MISC SALES TRAN,num_sales_in_SALAD BAR,num_sales_in_KIOSK-GAS,num_sales_in_ELECT &PLUMBING,num_sales_in_GRO BAKERY,num_sales_in_GM MERCH EXP,num_sales_in_FROZEN GROCERY,num_sales_in_COUP/STR & MFG,num_sales_in_SPIRITS,num_sales_in_GARDEN CENTER,num_sales_in_TOYS,num_sales_in_CHARITABLE CONT,num_sales_in_RESTAURANT,num_sales_in_RX,num_sales_in_PROD-WHS SALES,num_sales_in_MEAT-WHSE,num_sales_in_DAIRY DELI,num_sales_in_CHEF SHOPPE,num_sales_in_HBC,num_sales_in_DELI/SNACK BAR,num_sales_in_PORK,num_sales_in_AUTOMOTIVE,num_sales_in_VIDEO RENTAL,num_sales_in_CNTRL/STORE SUP,num_sales_in_HOUSEWARES,num_sales_in_POSTAL CENTER,num_sales_in_PHOTO,num_sales_in_VIDEO,num_sales_in_PHARMACY SUPPLY,median_sales_hour_for_user,mode_sales_day_for_user,mean_basket_check,n_items
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,2.569774,9.0,103.0,0.0,4.0,13.0,6.0,0.0,14.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,2.0,48.825714,109
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7,2.085000,8.0,94.0,0.0,1.0,7.0,3.0,0.0,8.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,6.0,37.530000,86
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8,2.472683,9.0,106.0,0.0,1.0,6.0,6.0,1.0,16.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,2.0,50.690000,116


In [19]:
# Подготовка описательных фичей (фичи для item_id)

def new_item_features(data, user_features, item_features):
    
    new_item_features = item_features.merge(data, on = 'item_id', how = 'left')
    new_user_features = user_features.merge(data, on = 'user_id', how = 'left')
    
    # Цена
    price = new_item_features.groupby('item_id')['sales_value'].sum() \
                            / new_item_features.groupby('item_id')['quantity'].sum()
    price = price.groupby('item_id').mean().reset_index()
    price.columns = ['item_id', 'price']
    price['price'].fillna(0, inplace = True)
    item_features = item_features.merge(price)

    # Среднее число покупок товара в неделю
    num_purchase_week = data.groupby(['item_id']).agg({'week_no': 'nunique', 'quantity': 'sum'}).reset_index()
    num_purchase_week['average_num_purchases_week'] = num_purchase_week['quantity'] / num_purchase_week['week_no']
    average_num_purchases_week = num_purchase_week[['item_id', 'average_num_purchases_week']]
    item_features = item_features.merge(average_num_purchases_week, on = 'item_id', how = 'left')
    item_features['average_num_purchases_week'].fillna(0, inplace= True)
    
    # Средняя цена товара в категории
    average_price_in_cat = item_features.groupby(['department'])['price'].mean().reset_index()
    average_price_in_cat.rename(columns = {'price': 'average_price_in_dep'}, inplace = True)
    average_price_in_cat.drop(labels = [0], axis = 0,inplace = True)
    item_features = item_features.merge(average_price_in_cat)
    
    # Цена товара/средняя цена товара в категории
    item_features['price/average_price_in_dep'] = item_features['price'] / item_features['average_price_in_dep']
    
    # Средняя цена товара в категории - Цена товара
    item_features['average_price_in_dep - item_price'] = item_features['average_price_in_dep'] - item_features['price']
    
    # Среднее время покупки товара
    new_item_features['hour'] = new_item_features['trans_time'] // 100
    hour = new_item_features.groupby('item_id')['hour'].median().reset_index()
    hour.columns = ['item_id', 'median_sales_hour_for_item']
    item_features = item_features.merge(hour, on = 'item_id', how = 'left')
    
    # Популярный день недели покупки товара
    new_item_features['weekday'] = new_item_features['day'] % 7
    week_day = new_item_features.groupby('item_id')['weekday'].apply(statistics.mode).reset_index()
    week_day.columns = ['item_id', 'mode_sales_day_for_item']
    item_features = item_features.merge(week_day, on = 'item_id', how = 'left')
    
    # Количество магазинов, в которых продается товар
    shops = new_item_features.groupby('item_id')['store_id'].nunique().reset_index()
    shops.columns = ['item_id', 'n_stores']
    item_features = item_features.merge(shops, on = 'item_id', how = 'left')
    
    item_features = item_features.replace(np.nan, 0)
    
    return item_features

In [20]:
%%time

item_features_new = new_item_features(data_train_lvl_2, user_features, item_features)
item_features_new.head(3)

CPU times: user 1.7 s, sys: 97.7 ms, total: 1.8 s
Wall time: 1.81 s


,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,price,average_num_purchases_week,average_price_in_dep,price/average_price_in_dep,average_price_in_dep - item_price,median_sales_hour_for_item,mode_sales_day_for_item,n_stores
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB,0.0,0.0,1.043754,0.0,1.043754,0.0,0.0,0
1,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ,0.0,0.0,1.043754,0.0,1.043754,0.0,0.0,0
2,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ,0.0,0.0,1.043754,0.0,1.043754,0.0,0.0,0


In [21]:
targets_lvl_2 = targets_lvl_2.merge(item_features_new, on = 'item_id', how = 'left')
targets_lvl_2 = targets_lvl_2.merge(user_features_new, on = 'user_id', how = 'left')

targets_lvl_2.head(3)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,price,average_num_purchases_week,average_price_in_dep,price/average_price_in_dep,average_price_in_dep - item_price,median_sales_hour_for_item,mode_sales_day_for_item,n_stores,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,average_sales_value,num_unique_department,num_sales_in_GROCERY,num_sales_in_MISC. TRANS.,num_sales_in_PASTRY,num_sales_in_DRUG GM,num_sales_in_MEAT-PCKGD,num_sales_in_SEAFOOD-PCKGD,num_sales_in_PRODUCE,num_sales_in_NUTRITION,num_sales_in_DELI,num_sales_in_COSMETICS,num_sales_in_MEAT,num_sales_in_FLORAL,num_sales_in_TRAVEL & LEISUR,num_sales_in_SEAFOOD,num_sales_in_MISC SALES TRAN,num_sales_in_SALAD BAR,num_sales_in_KIOSK-GAS,num_sales_in_ELECT &PLUMBING,num_sales_in_GRO BAKERY,num_sales_in_GM MERCH EXP,num_sales_in_FROZEN GROCERY,num_sales_in_COUP/STR & MFG,num_sales_in_SPIRITS,num_sales_in_GARDEN CENTER,num_sales_in_TOYS,num_sales_in_CHARITABLE CONT,num_sales_in_RESTAURANT,num_sales_in_RX,num_sales_in_PROD-WHS SALES,num_sales_in_MEAT-WHSE,num_sales_in_DAIRY DELI,num_sales_in_CHEF SHOPPE,num_sales_in_HBC,num_sales_in_DELI/SNACK BAR,num_sales_in_PORK,num_sales_in_AUTOMOTIVE,num_sales_in_VIDEO RENTAL,num_sales_in_CNTRL/STORE SUP,num_sales_in_HOUSEWARES,num_sales_in_POSTAL CENTER,num_sales_in_PHOTO,num_sales_in_VIDEO,num_sales_in_PHARMACY SUPPLY,median_sales_hour_for_user,mode_sales_day_for_user,mean_basket_check,n_items
0,2070,834103,1.0,2224,GROCERY,National,SOFT DRINKS,SFT DRNK SNGL SRV BTL CARB (EX,20 OZ,1.0,9.833333,1.043754,0.958081,0.043754,16.0,1.0,25,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,3.025931,10.0,213.0,0.0,4.0,31.0,11.0,1.0,16.0,0.0,3.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,17259.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,3.0,14.355581,156.0
1,2070,834103,1.0,2224,GROCERY,National,SOFT DRINKS,SFT DRNK SNGL SRV BTL CARB (EX,20 OZ,1.0,9.833333,1.043754,0.958081,0.043754,16.0,1.0,25,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,3.025931,10.0,213.0,0.0,4.0,31.0,11.0,1.0,16.0,0.0,3.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,17259.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,3.0,14.355581,156.0
2,2070,834103,1.0,2224,GROCERY,National,SOFT DRINKS,SFT DRNK SNGL SRV BTL CARB (EX,20 OZ,1.0,9.833333,1.043754,0.958081,0.043754,16.0,1.0,25,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,3.025931,10.0,213.0,0.0,4.0,31.0,11.0,1.0,16.0,0.0,3.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,17259.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,3.0,14.355581,156.0


In [22]:
# Данные для модели первого уровня

df_join_train_lvl_1 = pd.concat([data_train_lvl_1, data_val_lvl_1])
df_join_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0,1.50
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0,1.57


In [23]:
targets_lvl_2 = targets_lvl_2.merge(df_join_train_lvl_1.groupby(by = 'item_id').agg('sales_value').sum().rename('total_item_sales_value'), how='left',on='item_id')

targets_lvl_2  = targets_lvl_2.merge(df_join_train_lvl_1.groupby(by = 'item_id').agg('quantity').sum().rename('total_quantity_value'), how='left',on='item_id')

targets_lvl_2  = targets_lvl_2.merge(df_join_train_lvl_1.groupby(by = 'item_id').agg('user_id').count().rename('item_freq'), how='left',on='item_id')

targets_lvl_2  = targets_lvl_2.merge(df_join_train_lvl_1.groupby(by = 'user_id').agg('user_id').count().rename('user_freq'), how='left',on='user_id')

targets_lvl_2  = targets_lvl_2.merge(df_join_train_lvl_1.groupby(by = 'user_id').agg('sales_value').sum().rename('total_user_sales_value'), how='left',on='user_id')

targets_lvl_2  = targets_lvl_2.merge(df_join_train_lvl_1.groupby(by = 'item_id').agg('quantity').sum().rename('item_quantity_per_week')/df_join_train_lvl_1.week_no.nunique(), how='left',on='item_id')

targets_lvl_2  = targets_lvl_2.merge(df_join_train_lvl_1.groupby(by = 'user_id').agg('quantity').sum().rename('user_quantity_per_week')/df_join_train_lvl_1.week_no.nunique(), how='left',on='user_id')

targets_lvl_2 = targets_lvl_2.merge(df_join_train_lvl_1.groupby(by = 'item_id').agg('quantity').sum().rename('item_quantity_per_basket')/df_join_train_lvl_1.basket_id.nunique(), how='left',on='item_id')

targets_lvl_2 = targets_lvl_2.merge(df_join_train_lvl_1.groupby(by = 'user_id').agg('quantity').sum().rename('user_quantity_per_baskter')/df_join_train_lvl_1.basket_id.nunique(), how='left',on='user_id')

targets_lvl_2 = targets_lvl_2.merge(df_join_train_lvl_1.groupby(by = 'item_id').agg('user_id').count().rename('item_freq_per_basket')/df_join_train_lvl_1.basket_id.nunique(), how='left',on='item_id')

targets_lvl_2  = targets_lvl_2.merge(df_join_train_lvl_1.groupby(by = 'user_id').agg('user_id').count().rename('user_freq_per_basket')/df_join_train_lvl_1.basket_id.nunique(), how='left',on='user_id')

targets_lvl_2.head(3)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,price,average_num_purchases_week,average_price_in_dep,price/average_price_in_dep,average_price_in_dep - item_price,median_sales_hour_for_item,mode_sales_day_for_item,n_stores,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,average_sales_value,num_unique_department,num_sales_in_GROCERY,num_sales_in_MISC. TRANS.,num_sales_in_PASTRY,num_sales_in_DRUG GM,num_sales_in_MEAT-PCKGD,num_sales_in_SEAFOOD-PCKGD,num_sales_in_PRODUCE,num_sales_in_NUTRITION,num_sales_in_DELI,num_sales_in_COSMETICS,num_sales_in_MEAT,num_sales_in_FLORAL,num_sales_in_TRAVEL & LEISUR,num_sales_in_SEAFOOD,num_sales_in_MISC SALES TRAN,num_sales_in_SALAD BAR,num_sales_in_KIOSK-GAS,num_sales_in_ELECT &PLUMBING,num_sales_in_GRO BAKERY,num_sales_in_GM MERCH EXP,num_sales_in_FROZEN GROCERY,num_sales_in_COUP/STR & MFG,num_sales_in_SPIRITS,num_sales_in_GARDEN CENTER,num_sales_in_TOYS,num_sales_in_CHARITABLE CONT,num_sales_in_RESTAURANT,num_sales_in_RX,num_sales_in_PROD-WHS SALES,num_sales_in_MEAT-WHSE,num_sales_in_DAIRY DELI,num_sales_in_CHEF SHOPPE,num_sales_in_HBC,num_sales_in_DELI/SNACK BAR,num_sales_in_PORK,num_sales_in_AUTOMOTIVE,num_sales_in_VIDEO RENTAL,num_sales_in_CNTRL/STORE SUP,num_sales_in_HOUSEWARES,num_sales_in_POSTAL CENTER,num_sales_in_PHOTO,num_sales_in_VIDEO,num_sales_in_PHARMACY SUPPLY,median_sales_hour_for_user,mode_sales_day_for_user,mean_basket_check,n_items,total_item_sales_value,total_quantity_value,item_freq,user_freq,total_user_sales_value,item_quantity_per_week,user_quantity_per_week,item_quantity_per_basket,user_quantity_per_baskter,item_freq_per_basket,user_freq_per_basket
0,2070,834103,1.0,2224,GROCERY,National,SOFT DRINKS,SFT DRNK SNGL SRV BTL CARB (EX,20 OZ,1.0,9.833333,1.043754,0.958081,0.043754,16.0,1.0,25,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,3.025931,10.0,213.0,0.0,4.0,31.0,11.0,1.0,16.0,0.0,3.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,17259.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,3.0,14.355581,156.0,489.86,393,378,955,3599.87,4.318681,203.461538,0.002245,0.105787,0.00216,0.005456
1,2070,834103,1.0,2224,GROCERY,National,SOFT DRINKS,SFT DRNK SNGL SRV BTL CARB (EX,20 OZ,1.0,9.833333,1.043754,0.958081,0.043754,16.0,1.0,25,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,3.025931,10.0,213.0,0.0,4.0,31.0,11.0,1.0,16.0,0.0,3.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,17259.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,3.0,14.355581,156.0,489.86,393,378,955,3599.87,4.318681,203.461538,0.002245,0.105787,0.00216,0.005456
2,2070,834103,1.0,2224,GROCERY,National,SOFT DRINKS,SFT DRNK SNGL SRV BTL CARB (EX,20 OZ,1.0,9.833333,1.043754,0.958081,0.043754,16.0,1.0,25,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,3.025931,10.0,213.0,0.0,4.0,31.0,11.0,1.0,16.0,0.0,3.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,17259.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,3.0,14.355581,156.0,489.86,393,378,955,3599.87,4.318681,203.461538,0.002245,0.105787,0.00216,0.005456


In [24]:
# Обучение модели ранжирования

X_train = targets_lvl_2.drop('target', axis = 1)
y_train = targets_lvl_2[['target']]
X_train.shape, y_train.shape

((99835, 83), (99835, 1))

In [25]:
cat_feats = list(X_train.select_dtypes(include = ['object']).columns)
X_train[cat_feats] = X_train[cat_feats].astype('category')
cat_feats

['department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc']

In [26]:
%%time

lgb = LGBMClassifier(objective='binary',
                     num_boost_round = 10000,
                     n_jobs = 8,
                     force_row_wise = True,
                     seed = 24,
                     categorical_column = cat_feats,
                     verbose_eval = 1000)

lgb.fit(X_train, y_train)

train_preds = lgb.predict_proba(X_train)

[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
CPU times: user 9min 49s, sys: 6.89 s, total: 9min 56s
Wall time: 1min 25s


In [27]:
df_lvl_1_predict = targets_lvl_2.copy()

df_lvl_1_predict['proba_item_purchase'] = train_preds[:,1]

df_lvl_1_predict.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,price,average_num_purchases_week,average_price_in_dep,price/average_price_in_dep,average_price_in_dep - item_price,median_sales_hour_for_item,mode_sales_day_for_item,n_stores,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,average_sales_value,num_unique_department,num_sales_in_GROCERY,num_sales_in_MISC. TRANS.,num_sales_in_PASTRY,num_sales_in_DRUG GM,num_sales_in_MEAT-PCKGD,num_sales_in_SEAFOOD-PCKGD,num_sales_in_PRODUCE,num_sales_in_NUTRITION,num_sales_in_DELI,num_sales_in_COSMETICS,num_sales_in_MEAT,num_sales_in_FLORAL,num_sales_in_TRAVEL & LEISUR,num_sales_in_SEAFOOD,num_sales_in_MISC SALES TRAN,num_sales_in_SALAD BAR,num_sales_in_KIOSK-GAS,num_sales_in_ELECT &PLUMBING,num_sales_in_GRO BAKERY,num_sales_in_GM MERCH EXP,num_sales_in_FROZEN GROCERY,num_sales_in_COUP/STR & MFG,num_sales_in_SPIRITS,num_sales_in_GARDEN CENTER,num_sales_in_TOYS,num_sales_in_CHARITABLE CONT,num_sales_in_RESTAURANT,num_sales_in_RX,num_sales_in_PROD-WHS SALES,num_sales_in_MEAT-WHSE,num_sales_in_DAIRY DELI,num_sales_in_CHEF SHOPPE,num_sales_in_HBC,num_sales_in_DELI/SNACK BAR,num_sales_in_PORK,num_sales_in_AUTOMOTIVE,num_sales_in_VIDEO RENTAL,num_sales_in_CNTRL/STORE SUP,num_sales_in_HOUSEWARES,num_sales_in_POSTAL CENTER,num_sales_in_PHOTO,num_sales_in_VIDEO,num_sales_in_PHARMACY SUPPLY,median_sales_hour_for_user,mode_sales_day_for_user,mean_basket_check,n_items,total_item_sales_value,total_quantity_value,item_freq,user_freq,total_user_sales_value,item_quantity_per_week,user_quantity_per_week,item_quantity_per_basket,user_quantity_per_baskter,item_freq_per_basket,user_freq_per_basket,proba_item_purchase
0,2070,834103,1.0,2224,GROCERY,National,SOFT DRINKS,SFT DRNK SNGL SRV BTL CARB (EX,20 OZ,1.0,9.833333,1.043754,0.958081,0.043754,16.0,1.0,25,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,3.025931,10.0,213.0,0.0,4.0,31.0,11.0,1.0,16.0,0.0,3.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,17259.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,3.0,14.355581,156.0,489.86,393,378,955,3599.87,4.318681,203.461538,0.002245,0.105787,0.00216,0.005456,0.999938
1,2070,834103,1.0,2224,GROCERY,National,SOFT DRINKS,SFT DRNK SNGL SRV BTL CARB (EX,20 OZ,1.0,9.833333,1.043754,0.958081,0.043754,16.0,1.0,25,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,3.025931,10.0,213.0,0.0,4.0,31.0,11.0,1.0,16.0,0.0,3.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,17259.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,3.0,14.355581,156.0,489.86,393,378,955,3599.87,4.318681,203.461538,0.002245,0.105787,0.00216,0.005456,0.999938


In [28]:
# Оценка модели первого уровня

result_eval_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_eval_lvl_2.columns = ['user_id', 'actual']
result_eval_lvl_2.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."


In [29]:
%%time

result_eval_lvl_2['own_rec'] = result_eval_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N = 50))

CPU times: user 4.79 s, sys: 514 ms, total: 5.31 s
Wall time: 5.32 s


In [30]:
# Померяем precision только модели матчинга, чтобы понимать влияение ранжирования на метрики

sorted(calc_precision(result_eval_lvl_2, 5), key = lambda x: x[1], reverse = True)

[('own_rec', 0.16616187989556136)]

In [31]:
def rerank(user_id):
    
    return df_lvl_1_predict[df_lvl_1_predict['user_id'] == user_id].sort_values('proba_item_purchase', ascending = False).head(5).item_id.tolist()

In [32]:
result_eval_lvl_2['reranked_own_rec'] = result_eval_lvl_2['user_id'].apply(lambda user_id: rerank(user_id))
print(*sorted(calc_precision(result_eval_lvl_2, 5), key = lambda x: x[1], reverse = True), sep = '\n')

('own_rec', 0.16616187989556136)
('reranked_own_rec', 0.15519582245430807)


---

## Оценка на retail_test1

In [33]:
data_test = pd.read_csv('data/retail_test1.csv')
data_test = data_test[data_test['user_id'].isin(common_users)]
print_stats_data(data_test,'data_test')

data_test.head(2)

data_test
Shape: (83656, 12) Users: 1663 Items: 19981


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0


In [34]:
result_test = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_test.columns = ['user_id', 'actual']

result_test.head(2)

,user_id,actual
0,1,"[880007, 883616, 931136, 938004, 940947, 94726..."
1,6,"[956902, 960791, 1037863, 1119051, 1137688, 84..."


In [35]:
%%time

result_test['own_rec'] = result_test['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))

sorted(calc_precision(result_test, 5), key = lambda x: x[1], reverse = True)

CPU times: user 3.9 s, sys: 110 ms, total: 4.01 s
Wall time: 4.01 s


[('own_rec', 0.13674082982561636)]

In [36]:
result_test['reranked_own_rec'] = result_test['user_id'].apply(lambda user_id: rerank(user_id))

In [37]:
print(*sorted(calc_precision(result_test, 5), key = lambda x: x[1], reverse = True), sep = '\n')

('own_rec', 0.13674082982561636)
('reranked_own_rec', 0.1302465423932652)
